# AllSides sources & bias crawler

Get and save a list of rated news sources as left or right and in between.

A CSV file will be created with the following columns:

- Source
- Label
- Agree
- Disagree
- Publisher URL
- Publisher site

In [1]:
!ipython -m pip install aiohttp bs4 requests

]0;IPython: research/crawlersRequirement already satisfied: aiohttp in /Users/cmin/anaconda3/lib/python3.7/site-packages (3.6.2)


In [2]:
import asyncio
import csv
import logging
import re
import urllib.parse as urlparse

import aiohttp

import bs4
import requests

In [5]:
url_tpl = "https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_source_type_tid%5B3%5D=3&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=&customFilter=1&page={}"
html_parser = "html5lib"
csv_header = [
    "source",
    "label",
    "agree",
    "disagree",
    "publisher",
    "site",
]
dump_path = "media-bias.csv"
encoding = "utf-8"
skip_blocked_sites = True

verbose = True  # make it True to see debugging messages
level = logging.DEBUG if verbose else logging.INFO
logging.root.handlers.clear()
logging.basicConfig(
    format="%(levelname)s - %(name)s - %(asctime)s - %(message)s",
    level=level
)


async def get_soup(session, url):
    abs_url = urlparse.urljoin(url_tpl, url)
    text = await (await session.get(abs_url)).text()
#     resp.raise_for_status()
    soup = bs4.BeautifulSoup(text, html_parser)
    return soup


async def get_publisher_url(session, src_url, source_name):
#     import code; code.interact(local={**globals(), **locals()})
    logging.debug("Getting publisher's URL for %r.", source_name)
    soup = await get_soup(session, src_url)
    div = soup.find("div", class_="dynamic-grid")
    if not div:
        return None
    
    url = div.find("a").get("href").strip()
    parsed = urlparse.urlparse(url)
    if not parsed.netloc:
        return None
    
    return url, parsed.netloc


async def save_pages(bias_writer, csvfile):
    async with aiohttp.ClientSession() as session:
        page = 0  # custom page if you want
        while True:
            logging.info("Crawling page %d...", page)
            url = url_tpl.format(page)
            soup = await get_soup(session, url)
            
            pub_coros = []
            extras = []
            table = soup.find("table")
            if not table or "no record" in table.find("tbody").find("tr").text.lower():
                logging.info("Reached empty table -> end of results/pages.")
                break
                
            for row in table.find("tbody").find_all("tr"):
                src_a = row.find("td", class_="source-title").find("a")
                src_url = src_a.get("href")
                source_name = src_a.text
                label_alt = row.find("td", class_="views-field-field-bias-image").find("img").get("alt")
                label = label_alt.split(":")[-1].strip()
                feedback = row.find("td", class_="community-feedback")
                agree = int(feedback.find("span", class_="agree").text)
                disagree = int(feedback.find("span", class_="disagree").text)
                
                extras.append([source_name, label, agree, disagree])
#                 import code; code.interact(local={**globals(), **locals()})
                pub_coros.append(get_publisher_url(session, src_url, source_name))
            
            publisher_details_list = await asyncio.gather(*pub_coros)
            for idx, publisher_details in enumerate(publisher_details_list):
                if not publisher_details:
                    if skip_blocked_sites:
                        continue
                    else:
                        publisher_details = ("", "")

#                 print(source_name, label, f"{agree}/{disagree}")
                bias_writer.writerow(extras[idx] + list(publisher_details))

            page += 1
            csvfile.flush()


async def main():
    with open(dump_path, "w", newline="", encoding=encoding) as csvfile:
        bias_writer = csv.writer(csvfile)
        bias_writer.writerow(csv_header)
        await save_pages(bias_writer, csvfile)
        
        
await main()

INFO - root - 2020-03-22 22:38:04,926 - Crawling page 0...
DEBUG - root - 2020-03-22 22:38:07,878 - Getting publisher's URL for 'AARP'.
DEBUG - root - 2020-03-22 22:38:07,880 - Getting publisher's URL for 'ABC News'.
DEBUG - root - 2020-03-22 22:38:07,881 - Getting publisher's URL for 'Abridge News'.
DEBUG - root - 2020-03-22 22:38:07,882 - Getting publisher's URL for 'Accuracy in Media '.
DEBUG - root - 2020-03-22 22:38:07,885 - Getting publisher's URL for 'ACLU'.
DEBUG - root - 2020-03-22 22:38:07,886 - Getting publisher's URL for 'AJ+'.
DEBUG - root - 2020-03-22 22:38:07,892 - Getting publisher's URL for 'Al Cardenas'.
DEBUG - root - 2020-03-22 22:38:07,896 - Getting publisher's URL for 'Al Jazeera'.
DEBUG - root - 2020-03-22 22:38:07,898 - Getting publisher's URL for 'AllSides'.
DEBUG - root - 2020-03-22 22:38:07,900 - Getting publisher's URL for 'AllSides Community'.
DEBUG - root - 2020-03-22 22:38:07,904 - Getting publisher's URL for 'Allysia Finley (Wall Street Journal)'.
DEBUG 

INFO - root - 2020-03-22 22:38:26,198 - Crawling page 3...
DEBUG - root - 2020-03-22 22:38:29,056 - Getting publisher's URL for 'Chris Britt (cartoonist)'.
DEBUG - root - 2020-03-22 22:38:29,057 - Getting publisher's URL for 'Chris Ruddy'.
DEBUG - root - 2020-03-22 22:38:29,059 - Getting publisher's URL for 'Christian Science Monitor'.
DEBUG - root - 2020-03-22 22:38:29,060 - Getting publisher's URL for 'Christiane Amanpour'.
DEBUG - root - 2020-03-22 22:38:29,063 - Getting publisher's URL for 'Christianity Today'.
DEBUG - root - 2020-03-22 22:38:29,065 - Getting publisher's URL for 'Christopher Buskirk'.
DEBUG - root - 2020-03-22 22:38:29,066 - Getting publisher's URL for 'Chuck Asay (cartoonist)'.
DEBUG - root - 2020-03-22 22:38:29,068 - Getting publisher's URL for 'City Journal'.
DEBUG - root - 2020-03-22 22:38:29,071 - Getting publisher's URL for 'CivilPolitics.org'.
DEBUG - root - 2020-03-22 22:38:29,074 - Getting publisher's URL for 'Clay Bennett (cartoonist)'.
DEBUG - root - 202

DEBUG - root - 2020-03-22 22:38:50,523 - Getting publisher's URL for 'Forbes'.
DEBUG - root - 2020-03-22 22:38:50,524 - Getting publisher's URL for 'Foreign Affairs'.
DEBUG - root - 2020-03-22 22:38:50,527 - Getting publisher's URL for 'Foreign Policy'.
DEBUG - root - 2020-03-22 22:38:50,531 - Getting publisher's URL for 'Fox News Latino'.
DEBUG - root - 2020-03-22 22:38:50,535 - Getting publisher's URL for 'Fox News Opinion'.
DEBUG - root - 2020-03-22 22:38:50,539 - Getting publisher's URL for 'Fox Online News'.
DEBUG - root - 2020-03-22 22:38:50,540 - Getting publisher's URL for 'Frank Bruni'.
DEBUG - root - 2020-03-22 22:38:50,542 - Getting publisher's URL for 'Frank Miniter'.
DEBUG - root - 2020-03-22 22:38:50,546 - Getting publisher's URL for 'Freakonomics'.
DEBUG - root - 2020-03-22 22:38:50,550 - Getting publisher's URL for 'Freedom House'.
DEBUG - root - 2020-03-22 22:38:50,553 - Getting publisher's URL for 'Froma Harrop'.
DEBUG - root - 2020-03-22 22:38:50,557 - Getting publis

DEBUG - root - 2020-03-22 22:39:09,015 - Getting publisher's URL for 'Jonathan Miller'.
DEBUG - root - 2020-03-22 22:39:09,016 - Getting publisher's URL for "Journalist's Resource".
DEBUG - root - 2020-03-22 22:39:09,019 - Getting publisher's URL for 'Juan Williams'.
DEBUG - root - 2020-03-22 22:39:09,021 - Getting publisher's URL for 'Jubilee Media'.
DEBUG - root - 2020-03-22 22:39:09,023 - Getting publisher's URL for 'Judicial Watch'.
DEBUG - root - 2020-03-22 22:39:09,026 - Getting publisher's URL for 'Judy Sheindlin'.
DEBUG - root - 2020-03-22 22:39:09,030 - Getting publisher's URL for 'Julian Zelizer'.
DEBUG - root - 2020-03-22 22:39:09,033 - Getting publisher's URL for 'Kathleen Parker'.
DEBUG - root - 2020-03-22 22:39:09,035 - Getting publisher's URL for 'Katie Hill'.
DEBUG - root - 2020-03-22 22:39:09,037 - Getting publisher's URL for 'KATU'.
DEBUG - root - 2020-03-22 22:39:09,041 - Getting publisher's URL for 'Ken Catalino (cartoonist)'.
DEBUG - root - 2020-03-22 22:39:09,046 

DEBUG - root - 2020-03-22 22:39:29,923 - Getting publisher's URL for 'New York Magazine'.
DEBUG - root - 2020-03-22 22:39:29,926 - Getting publisher's URL for 'New York Post'.
DEBUG - root - 2020-03-22 22:39:29,929 - Getting publisher's URL for 'New York Times - News'.
DEBUG - root - 2020-03-22 22:39:29,932 - Getting publisher's URL for 'New York Times - Opinion'.
DEBUG - root - 2020-03-22 22:39:29,937 - Getting publisher's URL for 'NewsBusters'.
DEBUG - root - 2020-03-22 22:39:29,940 - Getting publisher's URL for 'Newsmax - News'.
DEBUG - root - 2020-03-22 22:39:29,944 - Getting publisher's URL for 'Newsmax - Opinion'.
DEBUG - root - 2020-03-22 22:39:29,948 - Getting publisher's URL for 'NewsOne'.
DEBUG - root - 2020-03-22 22:39:29,950 - Getting publisher's URL for 'Newsweek'.
DEBUG - root - 2020-03-22 22:39:29,953 - Getting publisher's URL for 'Newt Gingrich'.
DEBUG - root - 2020-03-22 22:39:29,955 - Getting publisher's URL for 'Nicholas Kristof'.
DEBUG - root - 2020-03-22 22:39:29,9

DEBUG - root - 2020-03-22 22:39:54,688 - Getting publisher's URL for 'Ryan Cooper'.
DEBUG - root - 2020-03-22 22:39:54,693 - Getting publisher's URL for 'S.E. Cupp'.
DEBUG - root - 2020-03-22 22:39:54,698 - Getting publisher's URL for 'Saagar Enjeti'.
DEBUG - root - 2020-03-22 22:39:54,700 - Getting publisher's URL for 'Sally Pipes'.
DEBUG - root - 2020-03-22 22:39:54,702 - Getting publisher's URL for 'Salon'.
DEBUG - root - 2020-03-22 22:39:54,704 - Getting publisher's URL for 'San Francisco Chronicle'.
DEBUG - root - 2020-03-22 22:39:54,705 - Getting publisher's URL for 'San Jose Mercury News'.
DEBUG - root - 2020-03-22 22:39:54,707 - Getting publisher's URL for 'SBG'.
DEBUG - root - 2020-03-22 22:39:54,709 - Getting publisher's URL for 'Science Daily'.
DEBUG - root - 2020-03-22 22:39:54,712 - Getting publisher's URL for 'Scientific American'.
DEBUG - root - 2020-03-22 22:39:54,716 - Getting publisher's URL for 'Scott Adams'.
DEBUG - root - 2020-03-22 22:39:54,719 - Getting publisher

DEBUG - root - 2020-03-22 22:40:17,062 - Getting publisher's URL for 'The Saturday Evening Post'.
DEBUG - root - 2020-03-22 22:40:17,067 - Getting publisher's URL for 'The Telegraph - UK'.
DEBUG - root - 2020-03-22 22:40:17,069 - Getting publisher's URL for 'The Texan'.
DEBUG - root - 2020-03-22 22:40:17,073 - Getting publisher's URL for 'The Texas Observer'.
DEBUG - root - 2020-03-22 22:40:17,074 - Getting publisher's URL for 'The Texas Tribune'.
DEBUG - root - 2020-03-22 22:40:17,076 - Getting publisher's URL for 'The Verge'.
DEBUG - root - 2020-03-22 22:40:17,078 - Getting publisher's URL for 'The Week - News'.
DEBUG - root - 2020-03-22 22:40:17,080 - Getting publisher's URL for 'The Week - Opinion'.
DEBUG - root - 2020-03-22 22:40:17,082 - Getting publisher's URL for 'The Weekly Standard'.
DEBUG - root - 2020-03-22 22:40:17,085 - Getting publisher's URL for 'The Western Journal'.
DEBUG - root - 2020-03-22 22:40:17,087 - Getting publisher's URL for 'TheBlaze.com'.
DEBUG - root - 202

Some publishers are blocked (no websites offered by AllSides), therefore fewer results in the CSV file.

Now let's find a good way of associating a side with a website in case multiple candidates are available.

In [6]:
side_dict = {}

with open(dump_path, newline="") as stream:
    reader = csv.reader(stream)
    print(next(reader))
    
    for row in reader:
        side_dict.setdefault(row[5], []).append((row[0], row[1], row[2]))

for site, sides in side_dict.items():
    if len(sides) > 1:
        print(site, sides)

['source', 'label', 'agree', 'disagree', 'publisher', 'site']
www.cnn.com [('CNN (Web News)', 'Lean Left', '30563'), ('CNN - Editorial', 'Left', '11793')]
www.thedailybeast.com [('Daily Beast', 'Left', '9638'), ('Newsweek', 'Lean Left', '2249')]
www.foxnews.com [('Fox News Opinion', 'Right', '9755'), ('Fox Online News', 'Lean Right', '25637')]
www.courier-journal.com [('Louisville Courier-Journal', 'Lean Left', '198'), ('The Courier-Journal', 'Lean Left', '134')]
www.nytimes.com [('New York Times - News', 'Lean Left', '17776'), ('New York Times - Opinion', 'Left', '4328')]
www.newsmax.com [('Newsmax - News', 'Lean Right', '7009'), ('Newsmax - Opinion', 'Right', '56')]
www.npr.org [('NPR Editorial ', 'Lean Left', '2717'), ('NPR Online News', 'Center', '21107')]
theweek.com [('The Week - News', 'Center', '3206'), ('The Week - Opinion', 'Lean Left', '37')]
online.wsj.com [('Wall Street Journal - Editorial', 'Lean Right', '5894'), ('Wall Street Journal - News', 'Center', '13773')]
